In [1]:
pip install mlxtend==0.23.1

  Obtaining dependency information for mlxtend==0.23.1 from https://files.pythonhosted.org/packages/1c/07/512f6a780239ad6ce06ce2aa7b4067583f5ddcfc7703a964a082c706a070/mlxtend-0.23.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.4 MB 3.6 MB/s eta 0:00:01
   --------- ------------------------------ 0.3/1.4 MB 3.5 MB/s eta 0:00:01
   ------------------ --------------------- 0.7/1.4 MB 4.8 MB/s eta 0:00:01
   ------------------------ --------------- 0.9/1.4 MB 5.7 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.4 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------  1.4/1.4 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------  1.4/1.4 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [3]:
# load the data set ans show the first five transaction
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [4]:
products = set()
for col in df.columns:
    products.update(df[col].unique())
print(products)

{'Cheese', 'Bagel', 'Wine', nan, 'Meat', 'Pencil', 'Bread', 'Diaper', 'Milk', 'Eggs'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [5]:
#create an itemset based on the products
# Convert rows to transactions, keeping NaN values
transactions = df.apply(lambda row: row.tolist(), axis=1).tolist()

# Handle NaN by replacing it with a special placeholder ('NaN')
transactions_with_nan = [
    ['NaN' if item is None else item for item in transaction]
    for transaction in transactions
]

# Flatten the transactions to create a list of unique items, including 'NaN'
# Convert all items to strings before sorting to avoid type comparison errors
all_items = sorted(set(str(item) for transaction in transactions_with_nan for item in transaction))

# Add 'NaN' to the list of items explicitly to handle it properly
# Since we've already converted everything to strings, we need to add 'nan' as a string
all_items.append('nan')

# encoding the feature
# Prepare the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit([[item] for item in all_items])

# Create dictionaries for each transaction
encoded_transactions = []
for transaction in transactions_with_nan:
    # Encode the current transaction
    encoded_array = encoder.transform([[item] for item in transaction])

    # Sum the encoded results to get the presence of each item
    summed_encoding = encoded_array.sum(axis=0)

    # Map the items back to the dictionary format
    encoded_dict = {item: int(value) for item, value in zip(encoder.categories_[0], summed_encoding)}
    encoded_transactions.append(encoded_dict)

# Display the first encoded transaction
encoded_transactions[0]

{'Bagel': 0,
 'Bread': 1,
 'Cheese': 1,
 'Diaper': 1,
 'Eggs': 1,
 'Meat': 1,
 'Milk': 0,
 'Pencil': 1,
 'Wine': 1,
 'nan': 0}

In [6]:
  # create new dataframe from the encoded features
df_new = pd.DataFrame(encoded_transactions)

  # show the new dataframe
df_new.head()

,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine,nan
0,0,1,1,1,1,1,0,1,1,0
1,0,1,1,1,0,1,1,1,1,0
2,0,0,1,0,1,1,1,0,1,0
3,0,0,1,0,1,1,1,0,1,0
4,0,0,0,0,0,1,0,1,1,0


In [7]:
# Since, the encoded dataframe consist of the empty column. We will drop the NaN column or u can use the index.
df_new.drop(columns='nan', inplace=True)

df_new.head()

,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,0,1,1,1,1,1,0,1,1
1,0,1,1,1,0,1,1,1,1
2,0,0,1,0,1,1,1,0,1
3,0,0,1,0,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [8]:
#Set threshold value untuk digunakan dalam penghitungan support
from mlxtend.frequent_patterns import apriori, association_rules

support = 0.2
frequent_itemsets = apriori(df_new, min_support=support, use_colnames=True)

frequent_itemsets

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.425397,(Bagel)
1,0.504762,(Bread)
2,0.501587,(Cheese)
3,0.406349,(Diaper)
4,0.438095,(Eggs)
5,0.476190,(Meat)
6,0.501587,(Milk)
7,0.361905,(Pencil)
8,0.438095,(Wine)
9,0.279365,"(Bagel, Bread)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [9]:
threshold = 0.6
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=threshold)

rules.drop(columns='zhangs_metric', inplace=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
1,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
2,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540
3,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891
4,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
5,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
6,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754
7,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624
8,"(Meat, Eggs)",(Cheese),0.266667,0.501587,0.215873,0.809524,1.613924,0.082116,2.616667
9,"(Cheese, Eggs)",(Meat),0.298413,0.476190,0.215873,0.723404,1.519149,0.073772,1.893773


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

## Association Rules: Analyzing Relationships Between Items Purchased Together

Association rules reveal the relationships between items that are frequently purchased together. The **antecedent** refers to the item(s) a customer buys first, while the **consequent** refers to the item(s) they are likely to buy next. The other columns describe various metrics used to assess the strength and significance of these associations.

### Key Metrics and Their Interpretation:

- **Antecedent Support**: This measures the frequency with which the antecedent (the "if" part of the rule) appears in the transactions. For example, in the first rule, Bagel appears in 42.54% of all transactions (0.425397), meaning Bagels are bought in about 42.5% of the total transactions.

- **Consequent Support**: This shows how frequently the consequent (the "then" part of the rule) appears in the transactions. For instance, Bread appears in 50.48% of transactions, indicating that Bread is bought in half of the transactions.

- **Support**: This value tells us how often the antecedent and consequent appear together in a transaction. For the rule **Bagel -> Bread**, the support is 27.94%, meaning that Bagel and Bread are purchased together in 27.94% of transactions. Support provides a measure of the overall importance of the rule.

- **Confidence**: Confidence indicates the likelihood that the consequent will appear given that the antecedent has appeared in a transaction. For the **Bagel -> Bread** rule, the confidence is 65.67%, meaning that if a customer buys Bagels, there is a 65.67% chance they will also buy Bread.

- **Lift**: Lift tells us how much more likely the consequent is to occur with the antecedent than without it. A lift greater than 1 suggests a positive relationship. In the case of **Bagel -> Bread**, the lift is 1.30, meaning that the occurrence of Bagel increases the likelihood of Bread being purchased by 1.30 times, compared to its overall likelihood of being purchased without the presence of Bagels.

- **Leverage**: Leverage measures the difference between the observed frequency of the rule and the expected frequency if the items were independent. For **Bagel -> Bread**, the leverage is 0.064641, which indicates the strength of the co-occurrence of Bagel and Bread relative to their individual probabilities.

- **Conviction**: This metric indicates the strength of the rule in terms of the likelihood of the antecedent occurring without the consequent. A conviction greater than 1 suggests a strong association. For **Bagel -> Bread**, the conviction is 1.44, which indicates a fairly strong association, meaning that if a customer buys Bagels, they are more likely to also buy Bread.

### Observations from the Rules:

- **Eggs -> Cheese** has a confidence of 68.12% and a lift of 1.36, meaning customers who buy Eggs are likely to buy Cheese with a strong relationship.
- **Cheese, Meat -> Eggs** has a confidence of 66.67% and a lift of 1.52, suggesting that the combination of Cheese and Meat strongly predicts the purchase of Eggs.
- **Milk -> Cheese** has a support of 30.48%, confidence of 60.76%, and a lift of 1.21, which means Milk and Cheese are bought together reasonably often, although the relationship is weaker compared to the other rules.

In general, these rules suggest that certain products are more likely to be purchased together, which can help in product placement, promotions, and targeted marketing strategies. For example, the strong relationship between Eggs and Cheese could be used for bundling them together in promotions or strategically placing them near each other on store shelves to encourage customers to purchase both.
